In [1]:
import numpy as np
import scipy.io
from midiutil import MIDIFile

In [2]:
key_time = 50e-3*0.5
Fs = 48000

In [3]:
keys = scipy.io.loadmat('MATLAB_data/ks_25Jan.mat')['key']
notes_list = scipy.io.loadmat('notes_88.mat')['notes']
freq_list = scipy.io.loadmat('freq_480.mat')['f_cropped']
#freq_list = scipy.io.loadmat('freq_441.mat')['f_cropped']
notes_list = notes_list.reshape(np.size(notes_list))
freq_list = freq_list.reshape(np.size(freq_list))

In [4]:
print(np.shape(notes_list))
print(np.shape(freq_list))
print(np.shape(keys))


(88,)
(1784,)
(1784, 207)


In [5]:
def get_intensity(cur_key,cur_freq,wave_freq):
    if cur_key == 1:
        sigma = notes_list[1]-notes_list[0]
    else:
        sigma = notes_list[cur_key-1]-notes_list[cur_key-2]
    sigma = sigma/2
    intensity = np.exp(-0.5 *((cur_freq-wave_freq)/sigma)**2)
    #print('cur', cur_freq,'wave',wave_freq)
    return intensity

In [6]:
def picknotes(cur_key,cur_freq):
    j = 0
    while j < len(cur_key):
        if cur_key[j] != 0:
            
            d = {}
            d['note'] = cur_key[j]
            d['start'] = j
            while cur_key[j] != 0 and j < len(cur_key)-1:
                j += 1
            d['end'] = j - 1
            intensity = get_intensity(cur_key[j-1],cur_freq,notes_list[cur_key[j-1]-1])
            d['intensity'] = intensity
            notes.append(d)
        j += 1
# picknotes(keys[500])     
# print(notes)      


In [7]:
notes = []
for i in range(np.shape(keys)[0]):
    cur_key = keys[i]
    cur_freq = freq_list[i]
    picknotes(cur_key,cur_freq)
    


In [8]:
notes = sorted(notes, key = lambda notes: notes["start"])

In [9]:
mf = MIDIFile(1)     # only 1 track
track = 0   # the only track

In [10]:
time = 0    # start at the beginning
mf.addTrackName(track, time, "Sample Track")
mf.addTempo(track, time, 1200)

In [11]:
channelmax = 15
channel = [(0,0)]*88

In [12]:
maxi=0

In [13]:
for note in notes:
    # if note['end']-note['start']+1 > maxi:
    #     maxi = note['end']-note['start']+1
    #mf.addNote(track,channel[note['note']-1],note['note'], note['start'], note['end']-note['start']+1,100)
    #if note['end']== note['start']:
    #    continue
    mf.addNote(track,1,note['note']+20, note['start'], note['end']-note['start']+1,50)
    #print(note['end']-note['start']+1)

In [15]:
with open("midifile/ori_ks_25Jan.mid", 'wb') as outf:
   mf.writeFile(outf)

In [ ]:
print(maxi)